In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [39]:
player_df = pd.read_csv("C:/Users/guygi/OneDrive/Bureau/concaf_analytics/datasets/clean/Player.csv", encoding='ISO-8859-1')
player_df = player_df.rename(columns={'ï»¿ShirtNumber': 'ShirtNumber'})
player_df

,ShirtNumber,PlayerName,Nationality,PlayerId,Position,Age,MarketValue,ClubId
0,6,Aaron Tshibola,Congo,243633,CM,29,450000,0
1,24,Abdallah Dipo Sima,Senegal,776798,RW,22,7000000,1
2,17,Abdallahi Mahmoud,Mauritania,615636,CM,23,400000,2
3,18,Abdelkabir Abqar,Maroc,559979,CB,24,3000000,3
4,16,Abdessamad Ezzalzouli,Maroc,724520,LW,22,15000000,4
...,...,...,...,...,...,...,...,...
614,2,Zephaniah Phiri,Zambia,1191910,RB,27,75000,375
615,26,Zidane Agustini Banjaqui,Guinea-Bissau,408963,AM,25,225000,376
616,24,Zineddine Belaid,Algeria,625142,CB,24,1400000,316
617,9,Zinho Gano,Guinea-Bissau,182446,CF,30,1200000,377


In [34]:
def scrap_cap_goals(player_id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
    }

    try:
        page = f"https://www.transfermarkt.fr/player-name/profil/spieler/{player_id}"
        response = requests.get(page, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes

        pageSoup = BeautifulSoup(response.content, 'html.parser')

        # Find the <a> tags and extract the text
        numbers = [a.text.strip() for a in pageSoup.find_all('a', class_='data-header__content data-header__content--highlight')]

        # Assign default values if caps and goals are not found
        caps, goals = (0, 0) if len(numbers) < 2 else map(int, numbers)

    except requests.RequestException as e:
        return f"Request error: {e}"
    except ValueError as e:
        return f"Data parsing error: {e}"

    return caps, goals

In [43]:
players_cap_goals = []
for i in range(player_df.shape[0]):
    player_id = player_df.iloc[i]['PlayerId']
    caps, goals = scrap_cap_goals(player_id)
    players_cap_goals.append({'PlayerId': player_id, 'Cap': caps, 'Goal': goals})

ValueError: not enough values to unpack (expected 2, got 0)

In [45]:
players_cap_goals = pd.DataFrame(players_cap_goals)

In [ ]:
players_df = players_df.merge(players_cap_goals, on='PlayerId', how='left')